In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from scipy.interpolate import CubicSpline
import tensorflow as tf

In [4]:
eeg_data = np.load('eeg_data_cleaned.npy')

# Reshape for LSTM (samples, timesteps, features)
X = np.transpose(eeg_data, (0, 2, 1))  # New shape: (11057, 256, 61)

# Load labels (assuming y_labels exists)
y = np.load('y.npy')


# Step 1: Split off 10% test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, stratify=y, random_state=42
)

# Step 2: Split remaining 90% into 80% train, 10% val
train_ratio = 8 / 9  # 80% out of the remaining 90%
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=(1 - train_ratio), stratify=y_train_val, random_state=42
)

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)



Train: (8845, 256, 61)
Val: (1106, 256, 61)
Test: (1106, 256, 61)


In [5]:
import numpy as np
from scipy.interpolate import CubicSpline

def add_noise(signal, noise_level=0.05):
    noise = np.random.normal(0, noise_level, signal.shape)
    return signal + noise

def time_warp(signal, warp_factor=0.2):
    orig_time = np.arange(signal.shape[0])
    warp_points = np.random.uniform(-warp_factor, warp_factor, 2)
    new_time = orig_time * (1 + warp_points[0]) + warp_points[1]
    warped = np.zeros_like(signal)
    for ch in range(signal.shape[1]):
        cs = CubicSpline(orig_time, signal[:, ch])
        warped[:, ch] = cs(new_time)
    return warped

def random_scaling(signal, scale_range=(0.8, 1.2)):
    scale = np.random.uniform(scale_range[0], scale_range[1])
    return signal * scale

def temporal_shift(signal, max_shift=10):
    shift = np.random.randint(-max_shift, max_shift)
    return np.roll(signal, shift, axis=0)

def window_slice(signal, window_ratio=0.8):
    seq_len = signal.shape[0]
    window_size = int(seq_len * window_ratio)
    start = np.random.randint(0, seq_len - window_size)
    window = signal[start:start+window_size, :]
    sliced = np.zeros_like(signal)
    for ch in range(signal.shape[1]):
        sliced[:, ch] = np.interp(
            np.linspace(0, 1, seq_len),
            np.linspace(0, 1, window_size),
            window[:, ch]
        )
    return sliced

def frequency_augmentation(signal, max_shift=5):
    augmented = np.zeros_like(signal)
    for ch in range(signal.shape[1]):
        fft = np.fft.rfft(signal[:, ch])
        shift = np.random.randint(-max_shift, max_shift)
        shifted_fft = np.roll(fft, shift)
        augmented[:, ch] = np.fft.irfft(shifted_fft, n=signal.shape[0])
    return augmented

def augment_EEG(X_batch):
    """Apply all augmentations in sequence to a batch."""
    augmented = []
    for x in X_batch:
        x_aug = x.copy()
        x_aug = add_noise(x_aug)
        x_aug = time_warp(x_aug)
        x_aug = random_scaling(x_aug)
        x_aug = temporal_shift(x_aug)
        x_aug = window_slice(x_aug)
        x_aug = frequency_augmentation(x_aug)
        augmented.append(x_aug)
    return np.array(augmented)


In [6]:
def create_augmented_dataset(X_original, y_original, num_augmentations=1):
    """
    Returns augmented dataset with guaranteed alignment
    :param num_augmentations: Number of augmented copies per sample
    """
    X_augmented = [X_original]
    y_augmented = [y_original]

    for _ in range(num_augmentations):
        augmented_X = augment_EEG(X_original)
        X_augmented.append(augmented_X)
        y_augmented.append(y_original)

    X_combined = np.concatenate(X_augmented, axis=0)
    y_combined = np.concatenate(y_augmented, axis=0)
    return X_combined, y_combined


In [7]:
def verify_alignment(X_original, X_augmented, y_original, y_augmented):
    """Verify X-y alignment after augmentation"""
    assert np.allclose(X_original, X_augmented[:len(X_original)]), "Original data corrupted!"
    assert np.array_equal(y_original, y_augmented[:len(y_original)]), "Original labels corrupted!"
    num_copies = len(X_augmented) // len(X_original) - 1
    for copy_idx in range(1, num_copies+1):
        start = copy_idx * len(X_original)
        end = (copy_idx+1) * len(X_original)
        assert np.array_equal(y_augmented[start:end], y_original), f"Copy {copy_idx} misaligned!"
    print("✅ All data aligned perfectly!")


In [8]:
# Example usage (replace with your actual data)
# X_train shape: (samples, timesteps, channels)
# y_train shape: (samples,)



# Create augmented dataset (e.g., double the data)
X_aug, y_aug = create_augmented_dataset(X_train, y_train, num_augmentations=1)
print(f"Original shape: {X_train.shape} → Augmented shape: {X_aug.shape}")

# Verify alignment
verify_alignment(X_train, X_aug, y_train, y_aug)


Original shape: (8845, 256, 61) → Augmented shape: (17690, 256, 61)
✅ All data aligned perfectly!


In [26]:

print(X_aug.shape)  # Output: (11057, 256, 61)
print(y_aug.shape)

(17690, 256, 61)
(17690,)


In [24]:
from tensorflow.keras import layers, models

input_shape = (256, 61)  # Timesteps, features (channels)

model_lstm_large_8255 = models.Sequential([
    layers.Input(shape=input_shape),
    
    # Bidirectional LSTM layers with residual connections
    layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Bidirectional(layers.LSTM(256)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    # Dense layers
    layers.Dense(512, activation='elu'),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    
    layers.Dense(1, activation='sigmoid')
])

model_lstm_smol = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Bidirectional(layers.LSTM(64)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='elu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [37]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define input shape (timesteps, features)
input_shape = (256, 61)

# Build the model
model_lstm_very_regularized = models.Sequential([
    layers.Input(shape=input_shape),
    layers.GaussianNoise(0.1),
    layers.Bidirectional(
        layers.LSTM(
            128,
            return_sequences=True,
            dropout=0.5,              # Input dropout
            recurrent_dropout=0,      # Must be 0 for cuDNN acceleration
            kernel_regularizer=regularizers.l2(1e-3)
        )
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    layers.Bidirectional(
        layers.LSTM(
            64,
            dropout=0.5,
            recurrent_dropout=0,
            kernel_regularizer=regularizers.l2(1e-3)
        )
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.6),
    layers.Dense(64, activation='elu', kernel_regularizer=regularizers.l2(1e-3)),
    layers.BatchNormalization(),
    layers.Dropout(0.7),
    layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras import layers, models, regularizers

input_shape = (256, 61)

model_lstm = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Bidirectional(
        layers.LSTM(
            128,
            return_sequences=True,
            dropout=0.3,              # Reduced dropout
            recurrent_dropout=0,      # cuDNN compatible
            kernel_regularizer=regularizers.l2(1e-4)  # Reduced L2
        )
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.4),             # Reduced dropout
    layers.Bidirectional(
        layers.LSTM(
            64,
            dropout=0.3,              # Reduced dropout
            recurrent_dropout=0,
            kernel_regularizer=regularizers.l2(1e-4)  # Reduced L2
        )
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.6),             # Reduced dropout
    layers.Dense(64, activation='elu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Dropout(0.6),             # Reduced dropout
    layers.Dense(1, activation='sigmoid')
])

model_lstm.summary()
# Compile the model
model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_83 (Bidirecti  (None, 256, 256)         194560    
 onal)                                                           
                                                                 
 batch_normalization_119 (Ba  (None, 256, 256)         1024      
 tchNormalization)                                               
                                                                 
 dropout_119 (Dropout)       (None, 256, 256)          0         
                                                                 
 bidirectional_84 (Bidirecti  (None, 128)              164352    
 onal)                                                           
                                                                 
 batch_normalization_120 (Ba  (None, 128)              512       
 tchNormalization)                                   

In [38]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Optional: Early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Fit the model
history = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


Epoch 1/100
277/277 [==============================] - 28s 86ms/step - loss: 0.9604 - accuracy: 0.5462 - val_loss: 0.7251 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 2/100
277/277 [==============================] - 27s 99ms/step - loss: 0.8039 - accuracy: 0.5916 - val_loss: 0.7009 - val_accuracy: 0.6772 - lr: 0.0010
Epoch 3/100
277/277 [==============================] - 27s 98ms/step - loss: 0.7428 - accuracy: 0.6256 - val_loss: 0.6800 - val_accuracy: 0.6926 - lr: 0.0010
Epoch 4/100
277/277 [==============================] - 27s 97ms/step - loss: 0.7134 - accuracy: 0.6495 - val_loss: 0.6805 - val_accuracy: 0.6736 - lr: 0.0010
Epoch 5/100
277/277 [==============================] - 27s 97ms/step - loss: 0.6755 - accuracy: 0.6836 - val_loss: 0.6307 - val_accuracy: 0.7297 - lr: 0.0010
Epoch 6/100
277/277 [==============================] - 27s 99ms/step - loss: 0.6409 - accuracy: 0.7178 - val_loss: 0.7580 - val_accuracy: 0.6790 - lr: 0.0010
Epoch 7/100
277/277 [==============================]

In [28]:
test_loss, test_acc = model_lstm.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


35/35 [==============================] - 1s 31ms/step - loss: 0.8378 - accuracy: 0.7161
Test accuracy: 0.7161


In [31]:
model_lstm.save('eeg_pure_lstm_model1.keras')